In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/My\ Drive/

/content/drive/My Drive


In [3]:

import sys
import numpy as np
import re
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import SimpleRNN
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import os
from nltk.tokenize import TweetTokenizer

from keras import backend as K
import imp
tknzr = TweetTokenizer()


Using TensorFlow backend.


In [4]:
train=[]
test=[]
with open('train1.txt', 'r') as lis:
    for i in lis:
        j = i[:-1]
        train.append(j)
lis.close()
with open('test1.txt', 'r') as lis:
    for i in lis:
        j = i[:-1]
        test.append(j)
lis.close()

nTrain=[]
nTest=[]
for i in train:
    nTrain.extend(tknzr.tokenize(i))
for i in test:
    nTest.extend(tknzr.tokenize(i))
    
raw_text = nTrain
# create mapping of unique chars to integers, and a reverse mapping
chars=set(raw_text)
print(len(chars))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 5
datax = []
datay = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	datax.append([char_to_int[char] for char in seq_in])
	datay.append(char_to_int[seq_out])

print(len(datax[0]),len(datax),len(datay))

dataX=datax[0:100000]
dataY=datay[0:100000]
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
y.shape

16090
Total Characters:  793174
Total Vocab:  16090
5 793169 793169
Total Patterns:  100000


(100000, 16086)

In [0]:
# define the model
model = Sequential()
model.add(SimpleRNN(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weight2.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=25, batch_size=64, callbacks=callbacks_list)

In [0]:

# load the network weights

filename = "weight2.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print(start)
print(pattern)

pre=" "
for i in range(100):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	pre=pre+result+" "
	seq_in = [int_to_char[value] for value in pattern]
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print(pre)
print("\nDone.")